# Windows Subsystem for Linux (WSL) lab🧪 for near-simultaneous🔁 Jupyter Notebook📓 development
<img src="https://www.game-experience.nl/wp-content/uploads/2018/04/Windows-10-logo-300x300.png" alt="Windows" width="40" height="40"> <img src="https://upload.wikimedia.org/wikipedia/commons/3/35/Tux.svg" alt="Linux" width="40" height="40"> <img src="https://jupyter.org/assets/main-logo.svg" alt="Jupyter" width="40" height="40">
------

* **Author:** Sevickson Kwidama 
    * [Twitter](https://twitter.com/SKwid345)
    * [LinkedIn](https://nl.linkedin.com/in/sevickson)
    * [GitHub Repo](https://github.com/sevickson/wsl_jupyter)
* **Version:** 1.0

This Jupyter Notebook gives you the possibility to very easily setup a WSL lab for developing cross-platform Notebooks.  
This is quite useful when you have a Notebook that has some CLI commands and you would like to check if those commands work on Windows and different Linux variants.

I use this Notebook in Visual Studio Code but it can also be used in Jupyter Lab without any issues.  
The reason it is near-simultaneous is because normally changes to textfiles or like .py files are immediately visible in the other instance of Visual Code when you use the `File -> Auto Save` function.   
Probably because a .ipynb Notebook is actually a JSON file that later is generated to become the Jupyter Notebook, the file needs to be reloaded, closed/opened, to see the changes.

## Load Functions and Imports *(Run Once)*
To be certain, I always start with an upgrade check of `pip`.  
And get the needed dependencies, if any, this only needs to be run once.  

On some systems *Python* can be called by using `python3` or `python`, this is the reason for the extra `which` check below.  
In `Requirements.txt` I put all the initial modules that need to be installed by `pip` for this Notebook.

In [ ]:
import shutil
if shutil.which("python3") is None:
    !python -m pip install --upgrade --user pip
    !python -m pip install --upgrade --user -r REQ\\Requirements.txt 
else:
    !python3 -m pip install --upgrade --user pip
    !python3 -m pip install --upgrade --user -r REQ\\Requirements.txt 

I put most of the Functions in a separate Notebook to make this original one shorter and more readable.  
By using `%run` I can import all the Functions and variables from the other Notebook in this one.

In [ ]:
%run REQ\\Imports_Functions.ipynb

## Install WSL *(Run Once)*
**BEWARE**: Below cells need to be run with Administrator priviledges. This can be done in Windows by opening the Notebook as Adminstrator.  

The cell below checks first if you are an Administrator, if you are it will run the needed steps to install WSL.  
If you are running Windows Build >= 19041 it will automatically also install WSL2.  
After WSL is installed a restart is needed before going forward.

In [ ]:
install_wsl()

If you get following message at the end: 
```
WSL 2 requires an update to its kernel component. For information please visit https://aka.ms/wsl2kernel
```
Please run the code below, when finished error should dissapear.

In [ ]:
update_wsl2()

## Install Linux Distributions
After installing WSL you can finally get to the more interesting parts, installation and use of Linux distributions.  

In this Notebook I only have *Ubuntu* and *Debian* as choices, as they are the most used distributions.  
If you need another distribution you can find the complete list in the [Windows Store] (https://aka.ms/wslstore) and direct links for setting in `REQ\WSL_VMs.csv` can be found [here](https://docs.microsoft.com/en-us/windows/wsl/install-manual).

If you already downloaded and installed the distribution in the past you only need to register. The check is done automatically below and if distribution is already available it will install it instead of downloading it again.
To make sure you have the latest versions you will need to update it through the Windows Store on your own system.

In [ ]:
@interact_manual
def get_WSL_VM(Distro=list(wsl_df.Name)):
    add_distro(Distro)

**BEWARE**: After running commands above the commands below can be run in a normal Jupyter Notebook.  
Below command is just to check which distributions are installed at the moment.

In [ ]:
!wsl --list --verbose

Based on the distributions installed below is a dropdown to choose which distribution will be used for the subsequent commands, following 2 cells are the dropdown and setting the distribution name in the variable `VM_Name`.

In [ ]:
distro_dd = interactive(get_installed, Distro=installed_distro())
display(distro_dd)

In [ ]:
VM_Name = distro_dd.result
VM_Name

### Setting the username and password used for the Linux distribution
`getpass` shows a pop-up window to set the password.  
The username and password set in the variables are used to create the user account in the Linux distribution.

In [ ]:
username = 'user_jupyter'
password = getpass.getpass()

## Configure Distribution
First always run updates and install needed packages like `sudo` and `openssl` to be able to create the user account. 

In [ ]:
install_update_packages(VM_Name)

Create user account based on the `username` and `password` set above. After this you can just use this to login to the distribution.

In [ ]:
create_user(VM_Name)

To be able to make this Jupyter lab work, Jupyter and all it's dependencies need to be installed.  

In [ ]:
install_jupyter(VM_Name)

Below is just a quick check to see if everything has been installed correctly.  
If everything is OK all commands will have a version number.

In [ ]:
!wsl -d {VM_Name} python3 --version
!wsl -d {VM_Name} pip3 --version
!wsl -d {VM_Name} jupyter --version

### Visual Basic Code
Setting this line of code in other Jupyter Notebooks you can easily start the Visual Basic in WSL pointing to the directory the Notebook is in.  
You will only need to set the `{VM_Name}` if you are using in another Notebook.

In [ ]:
!wsl -d {VM_Name} code .

Install **Python** extensions through "Extensions" and reload the Visual Basic Code in WSL.    
Now if you want to open an .ipynb it will ask for an interpreter, if you set the Python interpreter in your Linux environment you will be able to use it. 

### Jupyter Lab
If you want to run plain Jupyter Lab you can use the command below in a command line.  
You will need to change the `{VM_Name}` to the variable of the Linux distribution installed.
```
wsl -d {VM_Name} jupyter lab
```


## Uninstall Linux Distributions
When you uninstall/unregister the distribution it will not completely remove it.  
The benefit of this is that it makes it re-install easier, just run the **Install Linux Distributions** step.

In [ ]:
distro_dd_unr = interactive(get_installed, Distro=installed_distro())
display(distro_dd_unr)

In [ ]:
VM_Name_unr = distro_dd_unr.result
!wsl --unregister {VM_Name_unr}

References:
- https://docs.microsoft.com/en-us/windows/wsl/install-win10  
- https://medium.com/@harshityadav95/jupyter-notebook-in-windows-subsystem-for-linux-wsl-8b46fdf0a536  
- https://medium.com/@sayanghosh_49221/jupyter-notebook-in-windows-subsystem-for-linux-wsl-f075f7ec8691
- https://stackoverflow.com/questions/44996933/using-sudo-inside-jupyter-notebooks-cell  
- https://github.com/microsoft/WSL/issues/3284  
- https://docs.microsoft.com/en-us/windows/wsl/install-win10#update-to-wsl-2
- https://docs.microsoft.com/en-us/windows/wsl/install-manual
- https://www.tenforums.com/tutorials/128052-add-user-windows-subsystem-linux-wsl-distro-windows-10-a.html
- https://stackoverflow.com/questions/2150882/how-to-automatically-add-user-account-and-password-with-a-bash-script
- https://stackoverflow.com/questions/38883476/how-to-remove-those-x00-x00
- https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#interactive
